In [1]:
import PSO
import Elman
import copy
import torch
import numpy as np
import pandas as pd
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
import torch.utils.data as Data
# import torch.nn.functional as F

In [2]:
data = pd.read_csv('./train_data.csv')
test_data = pd.read_csv('./test_data.csv')

_data = pd.concat([data, test_data],axis =0).reset_index(drop=True)

In [3]:
sc_x_train = MinMaxScaler(feature_range=(0, 1))
sc_y_train = MinMaxScaler(feature_range=(0, 1))
_data_x = _data.drop(labels=['FF','FF1C','FF2C','Unnamed: 0'],axis=1).copy() 
sc_x_train.fit(_data_x)
_data_y = _data[['FF']].copy()
sc_y_train.fit(_data_y)

X = data.drop(labels=['FF','FF1C','FF2C','Unnamed: 0'],axis=1).copy() 
X_stand = sc_x_train.transform(X)
x_train = torch.tensor(data=X_stand)

Y = data[['FF']].copy()
Y_stand = sc_y_train.transform(Y)
y_train = torch.tensor(data=Y_stand)

In [19]:
X_test = test_data.drop(labels=['FF','FF1C','FF2C','Unnamed: 0'],axis=1).copy() 
X_stand_test = sc_x_train.transform(X_test)
x_test = torch.tensor(data=X_stand_test).type(torch.DoubleTensor)

Y_test = test_data[['FF']].copy()
Y_stand_test = sc_y_train.transform(Y_test)
y_test = torch.tensor(data=Y_stand_test).type(torch.DoubleTensor)

In [4]:
input_size, output_size = x_train.shape[1], y_train.shape[1]
hidden_size = 7

In [5]:
# 初始化权重矩阵
def initialize():
    w1 = torch.FloatTensor(input_size, hidden_size).type(torch.DoubleTensor)
#     init.normal_(w1,0,0.3)
#     init.normal(w1,0,0)
    w1 = Variable(w1, requires_grad=True)
    
    b1 = torch.FloatTensor(1, hidden_size).type(torch.DoubleTensor)
#     init.normal_(b1,0,0.3)
#     init.normal(w1,0,0)
    b1 = Variable(b1, requires_grad=True)

    w2 = torch.FloatTensor(hidden_size, output_size).type(torch.DoubleTensor)
#     init.normal_(w2,0,0.3)
#     init.normal(w2,0,0)
    w2 = Variable(w2, requires_grad=True)
    
    b2 = torch.FloatTensor(1, output_size).type(torch.DoubleTensor)
#     init.normal_(b2,0,0.3)
#     init.normal(w1,0,0)
    b2 = Variable(b2, requires_grad=True)

    w3 = torch.FloatTensor(hidden_size, hidden_size).type(torch.DoubleTensor)
#     init.normal_(w3,0,0.1)
#     init.normal(w2,0,0)
    w3 = Variable(w3, requires_grad=True)
    
    return w1,w2,w3,b1,b2

In [6]:
w1,w2,w3,b1,b2 = initialize()

In [7]:
size=3
w=0.9
c1=2
c2=2
max_v=0.5
max_iter=

# 编码长度
params_template = [copy.deepcopy(w1),copy.deepcopy(w2),copy.deepcopy(w3),copy.deepcopy(b1),copy.deepcopy(b2)]
chrom_len = 0
for i in range(len(params_template)):
    params_len = np.prod(params_template[i].shape)
    chrom_len += params_len

In [8]:
def chrom_to_params(chrom):
    """ 染色体转码为模型参数 """
    params_template = [copy.deepcopy(w1),copy.deepcopy(w2),
                   copy.deepcopy(w3),copy.deepcopy(b1),copy.deepcopy(b2)]
    params = copy.deepcopy(params_template)
    idx = 0
    for i in range(len(params)):
        params_len = np.prod(params[i].shape)
        param = torch.from_numpy(chrom[idx: idx+params_len]).reshape(params[i].shape)
        params[i] = param
        idx += params_len
    return params

In [11]:
def calculate_fitness(chrom):
    """ GA适应性函数 """
    fitness=[]
    elman = Elman.Elman(hidden_size,epochs=1,lr=0.1)
    
    for i in range(chrom.shape[0]):
        params = chrom_to_params(chrom[i])
        w1 = Variable(params[0], requires_grad=True)
        w2 = Variable(params[1], requires_grad=True)
        w3 = Variable(params[2], requires_grad=True)
        b1 = Variable(params[3], requires_grad=True)
        b2 = Variable(params[4], requires_grad=True)
        
        loss,_ = elman.train(x_train,y_train,w1,w2,w3,b1,b2)
    #     fitness_i = 1./loss
#         fitness.append(fitness_i)
        fitness.append(loss)
    return np.array(fitness)

In [15]:
elman = Elman.Elman(hidden_size,epochs=1,lr=0.1)
pso = PSO.PSO(calculate_fitness,size,w,c1,c2,max_v,chrom_len,max_iter)
best_pso_chrom = pso.roll()
best_pso_params = chrom_to_params(pso.roll())

In [16]:
w1,w2,w3,b1,b2 = best_pso_params[0],best_pso_params[1],best_pso_params[2],best_pso_params[3],best_pso_params[4]

In [24]:
test_loss, y_hat = elman.test(x_test,y_test,w1,w2,w3,b1,b2)